In [2]:
from model import *
from utils import *
import tensorflow as tf
import numpy as np
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags = tf.app.flags

flags.DEFINE_string('ASC', 'qb', 'restaurant or laptop or qb')
flags.DEFINE_string('DSC', 'yelp', '{yelp, twitter} for restaurant & {amazon, twitter} for laptop')
flags.DEFINE_integer('batch_size', 128, 'number of example per batch')
flags.DEFINE_float('learning_rate', 0.001, 'learning rate')
#flags.DEFINE_integer('n_iter', 25, 'training iteration')
flags.DEFINE_integer('n_iter', 3, 'training iteration')
# We slightly modify the training procedure. Feeding all DSC data in ONE epoch can get better results.
flags.DEFINE_float('gamma', 0.1, '{0.1, 0.1, 0.9, 0.2} for {res+yelp, res+twitter, laptop+amazon, laptop+twitter')
flags.DEFINE_integer('embedding_dim', 300, 'dimension of word embedding')
flags.DEFINE_integer('position_dim', 100, 'dimension of position embedding')
flags.DEFINE_integer('max_sentence_len', 85, 'max number of tokens per sentence')
flags.DEFINE_integer('max_target_len', 25, 'max number of tokens per target')
flags.DEFINE_float('keep_prob1', 0.5, 'dropout keep prob1')
flags.DEFINE_float('keep_prob2', 1.0, 'dropout keep prob2')
# Parameters for capsule layers.
flags.DEFINE_integer('filter_size', 3, 'filter_size')
flags.DEFINE_integer('sc_num', 16, 'sc_num')
flags.DEFINE_integer('sc_dim', 16, 'sc_dim')
flags.DEFINE_integer('cc_num',  3, 'cc_num')
flags.DEFINE_integer('cc_dim', 24, 'cc_dim')
flags.DEFINE_integer('iter_routing', 3, 'routing iteration')
flags.DEFINE_bool("reuse_embedding", True, "reuse word embedding & id, True or False")
FLAGS = flags.FLAGS

In [4]:
def main(_):
    start_time = time.time()
    info = ''
    index = 0
    
    for name, value in FLAGS.__flags.items():
        value = value.value
        if index < 19:
            info += '{}:{}  '.format(name, value)
        if index in [5, 11]:
            info += '\n'
        index += 1
    print('\n{:-^80}'.format('Parameters'))
    print(info + '\n')
    
    print('---------')
    print(FLAGS.ASC)
    
    data_path = 'data/{}/'.format(FLAGS.ASC)
    
    if not FLAGS.reuse_embedding :
        print('Initialize Word Dictionary & Embedding')
        word_dict = data_init(data_path, FLAGS.DSC)
        w2v = init_word_embeddings(data_path, word_dict, FLAGS.DSC)
    else:
        print('Reuse Word Dictionary & Embedding')
        with open(data_path + FLAGS.DSC + '_word2id.txt', 'r', encoding='utf-8') as f:
            word_dict = eval(f.read())
        w2v = np.load(data_path + FLAGS.DSC + '_word_embedding.npy')
    
    #print(w2v)
    #print(word_dict)
    
    model = MODEL(FLAGS, w2v, word_dict, data_path)
    model.run()
    
    end_time = time.time()
    print('Running Time: {:.0f}m {:.0f}s'.format((end_time-start_time) // 60, (end_time-start_time) % 60))

In [5]:
if __name__ == '__main__':
    tf.app.run()


-----------------------------------Parameters-----------------------------------
logtostderr:False  alsologtostderr:False  log_dir:  v:0  verbosity:0  stderrthreshold:fatal  
showprefixforinfo:True  run_with_pdb:False  pdb_post_mortem:False  run_with_profiling:False  profile_file:None  use_cprofile_for_profiling:True  
only_check_args:False  op_conversion_fallback_to_while_loop:False  test_random_seed:301  test_srcdir:  test_tmpdir:/tmp/absl_testing  test_randomize_ordering_seed:  xml_output_file:  

---------
qb
Reuse Word Dictionary & Embedding



W0319 06:55:36.539439 140331781224256 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:46: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W0319 06:55:36.554146 140331781224256 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:52: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



I am TransCap.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0319 06:55:36.588392 140331781224256 deprecation.py:506] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:65: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

W0319 06:55:40.536648 140331781224256 deprecation.py:323] From /home/rmajumder/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W0319 06:55:41.910882 140331781224256 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:124: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



W0319 06:55:41.926370 140331781224256 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb/model.py:125: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



[list([8, 7, 6, 5, 4, 3, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 5

ValueError: all input arrays must have the same shape